In [46]:
import pandas as pd
import datetime
import numpy as np
from preprocessing_functions.functions_preprocessing_alarms import first_eboxes_preprocess, first_lights_preprocess, big_preprocess_eboxes, big_preprocess_lights
from preprocessing_functions.functions_preprocessing_readings import split_readings, pre_power, pre_power_peak, get_indexes_split, pre_readings_batches
from preprocessing_functions.functions_preprocessing_meteo import first_meteo_preprocess, meteo_groupby
from preprocessing_functions.functions_joins import join_light_alarms_readings_meteo, join_eboxes_alarms_readings_meteo
from preprocessing_functions.utils_preprocessing import return_monday

In [47]:
pd.options.mode.chained_assignment = None

# Pipleine test:

In [ ]:
municiplality = "canyelles"

meteo = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/meteo_raw_data/meteo_{municiplality}.csv")
lights_alarms = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_lights_alarms.csv")
eboxes_alarms = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_eboxes_alarms.csv")
nodes = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_nodes.csv")
readings = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_readings.csv")

In [18]:
# Prepocess meteorological data:
meteo = first_meteo_preprocess(
    meteo = meteo,
    filter_dates = True,
    date_min = "2018-12-31",
    date_max = "2100-01-01"
)

meteo = meteo_groupby(meteo)

In [40]:
meteo

,Temp_max_max,Temp_avg_mean,Temp_avg_std,Temp_min_min,Dew_max_max,Dew_avg_mean,Dew_avg_std,Dew_min_min,Hum_max_max,Hum_avg_mean,...,Wind_avg_mean,Wind_avg_std,Wind_min_min,Pres_max_max,Pres_avg_mean,Pres_avg_std,Pres_min_min,Precipitation_mean,Precipitation_sum,dated
0,17.222222,8.785714,1.262969,2.777778,10.000000,2.634921,2.459729,-5.000000,93,67.485714,...,10.871429,0.930438,1,30.6,30.442857,0.097590,30.2,0.0,0.0,2015-01-01
1,17.222222,9.825397,1.715167,3.888889,12.222222,5.603175,2.086461,1.111111,94,75.728571,...,11.285714,3.068892,0,30.7,30.428571,0.179947,30.2,0.0,0.0,2015-01-08
2,15.000000,8.063492,1.462645,2.222222,12.222222,4.095238,2.499324,-1.111111,100,77.200000,...,9.785714,1.366783,0,30.2,29.885714,0.146385,29.7,0.0,0.0,2015-01-15
3,15.000000,8.500000,1.418934,2.222222,5.000000,-0.595238,3.009196,-13.888889,87,56.114286,...,11.271429,3.034092,0,30.3,30.057143,0.171825,29.7,0.0,0.0,2015-01-22
4,17.777778,8.777778,3.076193,2.222222,8.888889,2.111111,2.202692,-3.888889,93,65.085714,...,13.114286,3.745855,1,30.0,29.457143,0.214920,29.1,0.0,0.0,2015-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,18.888889,12.023810,1.523093,7.222222,13.888889,7.873016,1.477442,2.777778,100,76.500000,...,7.714286,1.145176,0,30.5,30.242857,0.127242,30.1,0.0,0.0,2022-12-29
418,17.222222,10.706349,1.953870,3.888889,12.222222,4.968254,1.631057,1.111111,88,69.385714,...,9.514286,1.475837,0,30.4,30.157143,0.171825,29.8,0.0,0.0,2023-01-05
419,16.111111,10.785714,1.563613,5.000000,12.222222,4.007937,3.081110,-7.222222,94,64.814286,...,12.400000,5.014645,0,30.4,29.900000,0.404145,29.2,0.0,0.0,2023-01-12
420,13.888889,6.619048,1.889006,-1.111111,6.111111,-2.388889,3.175912,-12.222222,81,54.871429,...,11.271429,3.436914,0,30.3,30.042857,0.151186,29.6,0.0,0.0,2023-01-19


In [19]:
# Readings preprocess:
powerReactive, powerActive, powerReactivePeak, powerActivePeak = split_readings(readings)
display(powerReactive)
powerReactive, powerActive = pre_power(powerReactive, "Reactive"), pre_power(powerActive, "Active")
display(powerReactive)
powerReactivePeak, powerActivePeak = pre_power_peak(powerReactivePeak, "Reactive"), pre_power_peak(powerActivePeak, "Active")

,id,dated,value,reading,component,componentid
1458542,AA000000D44C,2019-01-01 00:00:01,2370000,powerReactive,sum,0
1458543,AA000000D44C,2019-01-01 00:00:01,690000,powerReactive,p1,0
1458544,AA000000D44C,2019-01-01 00:00:01,1030000,powerReactive,p2,0
1458545,AA000000D44C,2019-01-01 00:00:01,650000,powerReactive,p3,0
1458546,AA000000D44C,2019-01-01 00:15:01,2340000,powerReactive,sum,0
...,...,...,...,...,...,...
7292687,AA000001169B,2022-10-06 07:30:01,0,powerReactive,p3,0
7292688,AA000001169B,2022-10-06 07:45:01,0,powerReactive,sum,0
7292689,AA000001169B,2022-10-06 07:45:01,0,powerReactive,p1,0
7292690,AA000001169B,2022-10-06 07:45:01,0,powerReactive,p2,0


,id,dated,powerReactive_sum,powerReactive_p1,powerReactive_p2,powerReactive_p3
0,AA000000D44C,2018-12-31,1.354618e+06,418489.583333,579010.416667,357100.694444
1,AA000000D44C,2019-01-07,1.411488e+06,457842.261905,567083.333333,387068.452381
2,AA000000D44C,2019-01-14,1.409673e+06,469687.500000,565535.714286,375208.333333
3,AA000000D44C,2019-01-21,1.384539e+06,455744.047619,554776.785714,373675.595238
4,AA000000D44C,2019-01-28,1.376741e+06,443898.809524,549122.023810,384404.761905
...,...,...,...,...,...,...
2162,AA000001169B,2022-09-05,6.023810e+01,51.928571,51.047619,71.095238
2163,AA000001169B,2022-09-12,4.861905e+01,37.547619,49.523810,76.785714
2164,AA000001169B,2022-09-19,5.313522e+01,41.485884,56.748886,68.255572
2165,AA000001169B,2022-09-26,5.760479e+01,40.455090,49.988024,68.910180


In [20]:
# Alarms preprocess
lights_alarms = first_lights_preprocess(lights_alarms)
eboxes_alarms = first_eboxes_preprocess(eboxes_alarms)

lights_alarms = big_preprocess_lights(lights_alarms)
eboxes_alarms = big_preprocess_eboxes(eboxes_alarms)

AA0000013912
AA0000013926
AA0000013931
AA000001393D
AA000001393E
AA0000016F53
AA000001726C
AA0000017324
AA00000184EB
AA0000018505
AA0000018696
AA0000018697
AA0000018698
AA0000018699
AA00000186B4
AA00000186D0
AA0000018704
AA0000018705
AA0000018706
AA0000018719
AA0000018745
AA0000018765
AA0000018769
AA000001876F
AA000001877F
AA0000018978
AA0000018981
AA00000189AC
AA00000189AD
AA00000189B8
AA00000189B9
AA00000189C6
AA00000189CB
AA00000189E7
AA0000018A0B
AA0000018A38
AA0000018A54
AA0000018A60
AA0000018A6C
AA0000018B4F
AA0000018B5F
AA0000018B99
AA0000018BA7
AA0000018BA8
AA0000018BB7
AA0000018C28
AA0000018C65
AA0000018C9B
AA0000018D53
AA0000018D6A
AA0000018D78
AA0000018D7B
AA0000018D85
AA0000018D87
AA0000018D89
AA0000018D8A
AA0000018D8C
AA0000018D93
AA0000018D9D
AA0000018DA1
AA0000018DA8
AA0000018DAB
AA0000018DB1
AA0000018DBD
AA0000018DD0
AA0000018DD8
AA0000018DDF
AA0000018DE3
AA0000018DEF
AA0000018DF7
AA0000018DFC
AA0000018E0C
AA0000018E10
AA0000018E9D
AA0000018EB4
AA0000018EC0
AA0000018ECB

In [21]:
# Join alarms, and nodes
lights_alarms_join = pd.merge(lights_alarms, nodes, on="id", how="left")

In [22]:
# You have to run the function join_eboxes_readings_meteo before runing join_light_alarms_readings_meteo
# because this last function modifies the datasets of the readings. In case you run the lights join
# first the code is going to return a Key error.
eboxes_alarms_join = join_eboxes_alarms_readings_meteo(
    eboxes_alarms = eboxes_alarms,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

lights_alarms_join = join_light_alarms_readings_meteo(
    light_errors = lights_alarms_join,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

In [23]:
print(len(lights_alarms_join))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(lights_alarms_join.isna().sum())

25993


id                                   0
week-4                               0
hours_week-4                         0
week-3                               0
hours_week-3                         0
week-2                               0
hours_week-2                         0
week-1                               0
hours_week-1                         0
current_week                         0
hours_current_week                   0
week+1                               0
hours_week+1                         0
week+2                               0
hours_week+2                         0
week+3                               0
hours_week+3                         0
week+4                               0
hours_week+4                         0
type                              2618
ebox_id                           2618
lat                               2618
lon                               2618
ReactivePeak_week-4               4488
ReactivePeak_week-3               4488
ReactivePeak_week-2      

In [24]:
print(len(eboxes_alarms_join))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(eboxes_alarms_join.isna().sum())

2321


id                                  0
week-4                              0
hours_week-4                        0
week-3                              0
hours_week-3                        0
week-2                              0
hours_week-2                        0
week-1                              0
hours_week-1                        0
current_week                        0
hours_current_week                  0
week+1                              0
hours_week+1                        0
week+2                              0
hours_week+2                        0
week+3                              0
hours_week+3                        0
week+4                              0
hours_week+4                        0
ReactivePeak_week-4               176
ReactivePeak_week-3               187
ReactivePeak_week-2               198
ReactivePeak_week-1               209
ReactivePeak_current_week         220
powerReactive_sum_week-4          176
powerReactive_p1_week-4           176
powerReactiv

# Readings spliting to be fit in memory

In [11]:
municipality = "illora"
data_dir = "/home/leibniz/Desktop/IHMAN"

readings_dates = pd.read_csv(f"{data_dir}/raw_data/{municipality}_readings_sorted.csv", usecols=["dated"]).reindex()

In [12]:
readings_dates["dated"] = pd.to_datetime(readings_dates["dated"])
start_date = return_monday(pd.to_datetime(readings_dates["dated"].min()))
end_date = return_monday(pd.to_datetime(readings_dates["dated"].max()) + pd.Timedelta(days=7))
range_dates = pd.date_range(start=start_date, end=end_date, freq="20W")

# Convert to timestamps and add the last date of the dataframe:
range_dates = [pd.Timestamp(return_monday(elem)) for elem in range_dates]
range_dates.append(pd.Timestamp(end_date))

In [13]:
# In this list we will store the indexes that will delimitate the batches:
spliting_indexes = np.array([])
for i in range(len(range_dates) -1):
    
    temp_df = readings_dates.loc[(range_dates[i] <= readings_dates["dated"]) & (readings_dates["dated"] < range_dates[i+1])]
    if len(temp_df) != 0:
        spliting_indexes = np.append(spliting_indexes, temp_df.index[-1])

# Add the index zero:
spliting_indexes = np.insert(spliting_indexes, 0, 0)

In [14]:
# Concatenated batches will be added to this dataframes:
powerReactive = pd.DataFrame()
powerActive = pd.DataFrame()
powerReactivePeak = pd.DataFrame()
powerActivePeak = pd.DataFrame()

# Name of the columns of the readings df:
readings_column_names = ["id", "dated", "value", "reading", "component", "componentid"]

# Convert to a list to iterate:
spliting_indexes_list = list(spliting_indexes)

# The readings preprocess must be done in batches because the datasets are too massive for some machines. In case that you 
# get a Memory error it is a good idea to take make larger the arg range_split_weeks_readings when running the code:
for i in range(len(spliting_indexes_list)-1):

    # For the first batch of date the function read_csv is able to read the names of the columns from the
    # first row of the csv file
    if i == 0:
        readings = pd.read_csv(
            f"{data_dir}/raw_data/{municipality}_readings_sorted.csv", 
            skiprows = int(spliting_indexes_list[i]), 
            nrows = int(spliting_indexes_list[i+1]-spliting_indexes_list[i]+1)
        )
    # For the other batches the function read_csv does not have acces to the column names so we have to
    # specify them
    else:
        readings = pd.read_csv(
            f"{data_dir}/raw_data/{municipality}_readings_sorted.csv",
            skiprows = int(spliting_indexes_list[i] + 2), 
            nrows = int(spliting_indexes_list[i+1]-spliting_indexes_list[i]),
            names = readings_column_names
        )

    powerReactive_, powerActive_, powerReactivePeak_, powerActivePeak_ = split_readings(readings)
    powerReactive_, powerActive_ = pre_power(powerReactive_, "Reactive"), pre_power(powerActive_, "Active")
    powerReactivePeak_, powerActivePeak_ = pre_power_peak(powerReactivePeak_, "Reactive"), pre_power_peak(powerActivePeak_, "Active")

    powerReactive = pd.concat([powerReactive, powerReactive_])
    powerActive = pd.concat([powerActive, powerActive_])
    powerReactivePeak = pd.concat([powerReactivePeak, powerReactivePeak_])
    powerActivePeak = pd.concat([powerActivePeak, powerActivePeak_])

In [4]:
meteo = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/meteo_raw_data/meteo_{municiplality}.csv")

NameError: name 'municiplality' is not defined

In [32]:
len(readings_dates)

42724532

# Output datasets tests:

In [14]:
out_boxes = pd.read_csv("preprocessing_results/out_boxes.csv")
out_lights = pd.read_csv("preprocessing_results/out_lights.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessing_results/out_boxes.csv'

In [13]:
mejorada = out_lights.loc[out_lights["location"] == "mejorada"]
len(mejorada)

192280

In [11]:
raw_eboxes_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_eboxes_alarms.csv")
raw_lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")

# raw_nodes = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_nodes.csv")
# raw_readings = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_readings.csv")
# raw_readings_sorted = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_readings_sorted.csv")

/tmp/ipykernel_155716/1412182631.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")


In [8]:
raw_meteo = pd.read_csv("/home/leibniz/Desktop/IHMAN/meteo_raw_data/new_meteo_mejorada.csv")

In [9]:
max(raw_meteo["Date"])

'2023-3-9'

In [10]:
min(raw_meteo["Date"])

'2014-1-1'

In [12]:
max(raw_lights_alarms["dated"])

'2036-01-21 21:49:44'

In [13]:
min(raw_lights_alarms["dated"])

'2014-09-10 18:51:55'

## alarms

In [26]:
raw_light_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")
len(raw_light_alarms)

/tmp/ipykernel_9059/2358804247.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_light_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")


5631094

In [27]:
first_lights_preprocess(raw_light_alarms)

,id,dated,alarm,flag
0,AA000000AB62,2015-03-24 18:22:12,lightcomm,off
1,AA000000AB62,2015-03-24 18:22:12,lighterr,off
2,AA000000AB63,2020-04-19 18:58:20,lighterr,on
3,AA000000AB63,2020-04-19 18:58:20,lighterr,on
4,AA000000AB63,2020-04-19 18:58:45,lighterr,off
...,...,...,...,...
196843,AA00000195F9,2023-04-02 18:43:16,lighterr,off
196844,AA00000195F9,2023-04-03 05:50:01,lighterr,on
196845,AA00000195F9,2023-04-03 05:50:01,lighterr,on
196846,AA00000195F9,2023-04-03 18:44:16,lighterr,off


In [1]:
test_date = "2022/06/09"

In [2]:
test_date[0:4]

'2022'

In [3]:
test_date[5:7]

'06'

In [5]:
test_date[8:]

'09'

In [1]:
from scrapy_meteo.scrapy_meteo import return_years_months

In [4]:
return_years_months(min_date="2014-04-01", max_date="2023-01-01")

4
2014
in!
[4, 5, 6, 7, 8, 9, 10, 11, 12]
2015
2016
2017
2018
2019
2020
2021
2022
2023


{2014: [4, 5, 6, 7, 8, 9, 10, 11, 12],
 2015: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2016: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2018: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2021: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2023: [1]}

In [3]:
from datetime import date
date.today().year

2023

In [4]:
date.today().month

5

In [5]:
import ast
ast.literal_eval("{'illora': ('2015-01-01', '2023-01-01'), 'canyelles': ('2015-01-01', '2023-01-01'), 'mejorada': ('2014-01-01', '2023-01-01')}")


{'illora': ('2015-01-01', '2023-01-01'),
 'canyelles': ('2015-01-01', '2023-01-01'),
 'mejorada': ('2014-01-01', '2023-01-01')}

In [18]:
meteo = pd.read_csv("/home/leibniz/Desktop/IHMAN/meteo_raw_data/new_meteo_mejorada.csv")
lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")

meteo = meteo_groupby(first_meteo_preprocess(meteo))
lights_alarms = 

/tmp/ipykernel_273039/815741648.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")


In [3]:
first_meteo_preprocess(meteo)

,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,Dew_min,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Pres_max,Pres_avg,Pres_min,Precipitation,Date
0,10.000000,8.277778,7.222222,8.888889,7.777778,6.111111,100,96.7,87,10,7.1,3,28.1,28.0,28.0,0.0,2014-01-01
1,12.777778,11.000000,8.888889,12.222222,10.277778,8.888889,100,96.2,94,18,11.7,6,28.0,27.9,27.9,0.0,2014-01-02
2,12.777778,11.944444,11.111111,12.222222,10.444444,8.888889,100,91.1,82,15,10.2,3,28.1,28.1,28.0,0.0,2014-01-03
3,11.111111,8.722222,5.000000,11.111111,4.666667,-17.777778,100,81.1,57,26,16.4,5,28.1,27.9,27.8,0.0,2014-01-04
4,8.888889,6.388889,3.888889,7.777778,3.666667,-17.777778,100,86.5,65,14,7.8,5,28.2,28.2,28.1,0.0,2014-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372,20.000000,12.611111,5.000000,0.000000,-1.666667,-6.111111,70,41.2,17,7,2.8,0,28.1,28.1,28.0,0.0,2023-03-27
3373,23.888889,13.611111,2.777778,0.000000,-2.722222,-5.000000,70,36.7,14,7,3.1,0,28.1,28.1,28.1,0.0,2023-03-28
3374,26.111111,16.111111,6.111111,3.888889,1.277778,0.000000,66,40.8,18,13,5.3,1,28.1,28.1,28.0,0.0,2023-03-29
3375,22.777778,16.944444,7.777778,8.888889,4.777778,1.111111,71,46.4,27,22,10.1,0,28.1,28.1,28.0,0.0,2023-03-30


In [6]:
meteo

,Unnamed: 0,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,Dew_min,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Pres_max,Pres_avg,Pres_min,Precipitation,Date
0,0,50,46.9,45,48,46.0,43,100,96.7,87,10,7.1,3,28.1,28.0,28.0,0.0,2014-1-1
1,1,55,51.8,48,54,50.5,48,100,96.2,94,18,11.7,6,28.0,27.9,27.9,0.0,2014-1-2
2,2,55,53.5,52,54,50.8,48,100,91.1,82,15,10.2,3,28.1,28.1,28.0,0.0,2014-1-3
3,3,52,47.7,41,52,40.4,0,100,81.1,57,26,16.4,5,28.1,27.9,27.8,0.0,2014-1-4
4,4,48,43.5,39,46,38.6,0,100,86.5,65,14,7.8,5,28.2,28.2,28.1,0.0,2014-1-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372,26,68,54.7,41,32,29.0,21,70,41.2,17,7,2.8,0,28.1,28.1,28.0,0.0,2023-3-27
3373,27,75,56.5,37,32,27.1,23,70,36.7,14,7,3.1,0,28.1,28.1,28.1,0.0,2023-3-28
3374,28,79,61.0,43,39,34.3,32,66,40.8,18,13,5.3,1,28.1,28.1,28.0,0.0,2023-3-29
3375,29,73,62.5,46,48,40.6,34,71,46.4,27,22,10.1,0,28.1,28.1,28.0,0.0,2023-3-30


In [34]:
lights_alarms["dated"].min()

'2014-09-10 18:51:55'

In [35]:
lights_alarms["dated"].max()

'2036-01-21 21:49:44'

In [22]:
preprocessed_lights = pd.read_csv("/home/leibniz/Desktop/IHMAN/preprocessing_results/out_lights.csv")

In [23]:
len(preprocessed_lights)

314413

In [24]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(preprocessed_lights.isna().sum())

Unnamed: 0                             0
id                                     0
week-4                                 0
hours_week-4                           0
week-3                                 0
hours_week-3                           0
week-2                                 0
hours_week-2                           0
week-1                                 0
hours_week-1                           0
current_week                           0
hours_current_week                     0
week+1                                 0
hours_week+1                           0
week+2                                 0
hours_week+2                           0
week+3                                 0
hours_week+3                           0
week+4                                 0
hours_week+4                           0
type                               48642
ebox_id                            57189
lat                                48642
lon                                48642
ReactivePeak_wee

In [8]:
clean = preprocessed_lights.loc[preprocessed_lights["Precipitation_sum_current_week"].isna()]

In [9]:
clean["current_week"].min()

'2014-10-06'

In [10]:
clean["current_week"].max()

'2015-06-01'

In [17]:
clean["current_week"].min()

'2014-10-06'

# Fuck it! Let's go step by step to see where the code is failing:

In [48]:
data_dir = "/home/leibniz/Desktop/IHMAN"
municipality = "canyelles"
n_weeks = 20

In [49]:
meteo = pd.read_csv(f"{data_dir}/meteo_raw_data/new_meteo_{municipality}.csv")
lights_alarms = pd.read_csv(f"{data_dir}/raw_data/{municipality}_lights_alarms.csv", usecols=lambda column: column != "info_value")
eboxes_alarms = pd.read_csv(f"{data_dir}/raw_data/{municipality}_eboxes_alarms.csv")
nodes = pd.read_csv(f"{data_dir}/raw_data/{municipality}_nodes.csv")
splitting_indexes = get_indexes_split(municipality, data_dir, n_weeks)

In [50]:
print(meteo["Date"].min())
print(meteo["Date"].max())
meteo = first_meteo_preprocess(meteo=meteo, filter_dates=True, date_min="2014-01-01", date_max="2023-12-31")
print(meteo["Date"].min())
print(meteo["Date"].max())
meteo = meteo_groupby(meteo)
print(meteo["dated"].min())
print(meteo["dated"].max())

2015-1-1
2023-1-9
2015-01-01 00:00:00
2023-01-31 00:00:00
2015-01-01 00:00:00
2023-01-26 00:00:00


In [51]:
powerReactive, powerActive, powerReactivePeak, powerActivePeak = pre_readings_batches(splitting_indexes, municipality, data_dir)

In [52]:
lights_alarms = first_lights_preprocess(lights_alarms)
eboxes_alarms = first_eboxes_preprocess(eboxes_alarms)

In [53]:
lights_alarms = big_preprocess_lights(lights_alarms)
eboxes_alarms = big_preprocess_eboxes(eboxes_alarms)

AA0000013912
AA0000013926
AA0000013931
AA000001393D
AA000001393E
AA0000016F53
AA000001726C
AA0000017324
AA00000184EB
AA0000018505
AA0000018696
AA0000018697
AA0000018698
AA0000018699
AA00000186B4
AA00000186D0
AA0000018704
AA0000018705
AA0000018706
AA0000018719
AA0000018745
AA0000018765
AA0000018769
AA000001876F
AA000001877F
AA0000018978
AA0000018981
AA00000189AC
AA00000189AD
AA00000189B8
AA00000189B9
AA00000189C6
AA00000189CB
AA00000189E7
AA0000018A0B
AA0000018A38
AA0000018A54
AA0000018A60
AA0000018A6C
AA0000018B4F
AA0000018B5F
AA0000018B99
AA0000018BA7
AA0000018BA8
AA0000018BB7
AA0000018C28
AA0000018C65
AA0000018C9B
AA0000018D53
AA0000018D6A
AA0000018D78
AA0000018D7B
AA0000018D85
AA0000018D87
AA0000018D89
AA0000018D8A
AA0000018D8C
AA0000018D93
AA0000018D9D
AA0000018DA1
AA0000018DA8
AA0000018DAB
AA0000018DB1
AA0000018DBD
AA0000018DD0
AA0000018DD8
AA0000018DDF
AA0000018DE3
AA0000018DEF
AA0000018DF7
AA0000018DFC
AA0000018E0C
AA0000018E10
AA0000018E9D
AA0000018EB4
AA0000018EC0
AA0000018ECB

In [54]:
lights_alarms = pd.merge(lights_alarms, nodes, on="id", how="left")

In [ ]:
meteo["Date"] = meteo["Date"].apply(return_monday())

In [55]:
lights_alarms = join_light_alarms_readings_meteo(
    light_errors = lights_alarms,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

In [56]:
lights_alarms.isna().sum()

id                                    0
week-4                                0
hours_week-4                          0
week-3                                0
hours_week-3                          0
                                  ...  
Pres_avg_avg_current_week         25993
Pres_avg_std_current_week         25993
Pres_min_min_current_week         25993
Precipitation_avg_current_week    25993
Precipitation_sum_current_week    25993
Length: 183, dtype: int64

In [57]:
len(lights_alarms)

25993